# LLM for Recommendation System - RAG

## TABLE OF CONTENT
### $~~~$ - 1. Load Tokenizer and Model from HuggingFace
### $~~~$ - 2. Load Vector Database
### $~~~$ - 3. Recommendation System

---
## 1. Load Tokenizer and Model from HuggingFace

In [ ]:
# Check Python vision
!python -V
# Check CUDA vision
!nvcc --version

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from getpass import getpass
import torch
import os

In [ ]:
# Check for GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available else "cpu")
#device = 'cpu' # Set to cpu when debugging
print(f"Using device: {device}")

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
access_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = access_token

In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
# model_id = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("[*] Tokenizer loaded.")

In [ ]:
# Load Model
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    token=access_token,
    # quantization_config=bnb_config,
).to(device)
print("[*] Model loaded.")

---
## 2. Load Embedding Model and Vector Database

### Embedding Model

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    multi_process=True,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

### Vector Database

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
base_dir = "../.."

In [ ]:
vector_db_dir = os.path.join(base_dir, 'Vector_DB')
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(
    vector_db_dir,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

### Test Query

In [ ]:
from time import time 
import pandas as pd
import random

In [ ]:
formatted_df = pd.read_csv(os.path.join(base_dir, 'trainData/amazon_products.train.formatted.csv'))

In [ ]:
def retrieve_product_information(df, query_value):
    product_index = df.index[df['PRODUCT_ID'] == query_value].tolist()[0]
    full_text = formatted_df.loc[product_index, 'TEXT']
    print(f'[*] Retrieved product full content:\n{full_text}')

    return formatted_df.loc[product_index, 'DESCRIPTION'], full_text

In [ ]:
random.seed(time())
random_product_id = random.choice(formatted_df['PRODUCT_ID'])
test_description, full_text = retrieve_product_information(formatted_df, random_product_id)

In [ ]:
print(f"[*] Starting retrieval for description:\n{test_description=}\n")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=test_description, k=6)[1:] # The first one will always be the qurey one, so skip it.
print("==================================Top document==================================")
print(retrieved_docs[0].page_content)
print("====================================Full Content====================================")
print(retrieved_docs[0].metadata['text'])

---
## 3. Recommendation System

In [ ]:
from transformers import pipeline

In [ ]:
Rec_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=1000,
    device=device
)

In [ ]:
# Test
Q = "What is 4+4? Answer:"
A = Rec_LLM(Q)
print(f'[*] {Q}{A[0]['generated_text']}')

### Prompt Template

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in context, give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Information of recommended products must be correct and matched in context, do not falsify information.
If the answer cannot be deduced from the context, do not give an answer.

Response must include product id, title, and reason for recommendation.
Response must strictly follow the template below:
i. - Product ID: <product id>: 
   - Title: <title>
   - Reason: <reason for recommendation>""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

### Recommendation Test

In [ ]:
random.seed(time())
random_product_id = random.choice(formatted_df['PRODUCT_ID'])
test_description, full_text = retrieve_product_information(formatted_df, random_product_id)

In [ ]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=test_description, k=11)[1:] # The first one will always be the qurey one, so skip it.

In [ ]:
retrieved_docs_text = [
    doc.metadata['text'] for doc in retrieved_docs
]  # We only need the text of the documents

In [ ]:
context = "\nExtracted products:"
context += "".join(
    [f"\n\nProduct {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
)

In [ ]:
final_prompt = RAG_PROMPT_TEMPLATE.format(
    question="Base on this product, recommend 5 best products from Context.", context=context
)

In [ ]:
# Redact an answer
recommedations = Rec_LLM(final_prompt)[0]["generated_text"]
print(recommedations)

In [ ]:
print(final_prompt)